In [1]:
import os 
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = '-1' # for using only cpu
import tensorflow as tf
import numpy as np

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST/', one_hot=True)

Extracting MNIST/train-images-idx3-ubyte.gz
Extracting MNIST/train-labels-idx1-ubyte.gz
Extracting MNIST/t10k-images-idx3-ubyte.gz
Extracting MNIST/t10k-labels-idx1-ubyte.gz


In [3]:
images = tf.placeholder(shape=[None, 784], dtype=tf.float32)
reshape_images = tf.reshape(images, [-1, 28, 28, 1])
labels = tf.placeholder(shape=[None, 10], dtype=tf.float32)

training = tf.placeholder(tf.bool)

In [5]:
# Model Architecture(Network)
conv1 = tf.layers.conv2d(reshape_images, 32, [3, 3], activation=tf.nn.relu)
pool1 = tf.layers.max_pooling2d(conv1, [3, 3], [2, 2])

conv2 = tf.layers.conv2d(pool1, 64, [3, 3], activation=tf.nn.relu)
pool2 = tf.layers.max_pooling2d(conv2, [3, 3], [2, 2])

flatten_feature = tf.contrib.layers.flatten(pool2)
dense1 = tf.layers.dense(flatten_feature, 1024, activation=tf.nn.relu)
drop1 = tf.layers.dropout(dense1, 0.5, training=training)

logits = tf.layers.dense(drop1, 10, activation=None)
cost = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels)

LEARNING_RATE = 1e-4
opt = tf.train.AdamOptimizer(LEARNING_RATE).minimize(cost)

각 함수 parameter

tf.layers.conv2d(input_vector, output_channel, [kernel_width, kernel_height], ...)

tf.layers.max_pooling2d(input_vector, [kernel_width, kernel_height], [strides_along_width, strides_along_height], ...)

tf.layers.dense(input_vector, output_channel, ...)

tf.layers.dropout(input_vector, drop_ratio, training, ...)

In [6]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [7]:
MAX_EPOCH = 10
BATCH_SIZE = 64
total_batch = int(mnist.train.num_examples/BATCH_SIZE)
total_batch

859

In [8]:
total_loss = 0
for epoch in range(MAX_EPOCH):    
    for i in range(total_batch):
        batch = mnist.train.next_batch(batch_size=BATCH_SIZE, shuffle=True)
        _, loss = sess.run([opt, cost], feed_dict={images: batch[0], labels: batch[1], training: True})
    
        if i % 100 == 0:
            test_acc = sess.run(cost,
                 feed_dict={images: mnist.test.images, labels: mnist.test.labels, training:False}
            )
            print('Test accuracy: {}'.format(test_acc))
        total_loss += loss
    print('{} epochs completed'.format(epoch))

Test accuracy: [ 2.26618767  2.51381183  2.3304944  ...,  2.36248589  2.47254705
  2.25476646]
Test accuracy: [ 0.19004999  0.7404108   0.12458982 ...,  0.84763896  1.49897373
  0.79925632]
Test accuracy: [ 0.00415652  0.28783059  0.01576156 ...,  0.17863193  0.45476639
  0.06421856]
Test accuracy: [ 0.00066652  0.05786695  0.00508397 ...,  0.02074625  0.22643431
  0.00784578]
Test accuracy: [ 0.00050091  0.01817501  0.00484115 ...,  0.02794873  0.08017522
  0.00562952]
Test accuracy: [ 0.00033742  0.01405802  0.00425066 ...,  0.01462842  0.09017809
  0.00371089]
Test accuracy: [ 0.00045051  0.00855944  0.00317771 ...,  0.00952792  0.03295648
  0.00340727]
Test accuracy: [ 0.00012242  0.0098108   0.00216892 ...,  0.00364984  0.07420675
  0.00279679]
Test accuracy: [ 0.00011503  0.0082651   0.0034858  ...,  0.00388522  0.04897175
  0.00216298]
Epoch: 0 completed
Test accuracy: [ 0.00027021  0.00419309  0.00369105 ...,  0.00309798  0.03848245
  0.00097906]
Test accuracy: [ 0.00010728  0.

KeyboardInterrupt: 